In [5]:
# library
import tensorflow as tf
import numpy as np,sys
import matplotlib.pyplot as plt
tf.set_random_seed(6789)
np.random.seed(678)

In [8]:
# create the normal distribution
def det(array_or_scalar):
    if array_or_scalar.size > 1:
        return np.linalg.det(array_or_scalar)
    else:
        return array_or_scalar
def get_h_mvn(x):

    """
    Computes the entropy of a multivariate Gaussian distribution:
    H(X) = (1/2) * log((2 * pi * e)^d * det(cov(X)))
    Arguments:
    ----------
    x: (n, d) ndarray
        n samples from a d-dimensional multivariate normal distribution
    Returns:
    --------
    h: float
        entropy H(X)
    """

    d = x.shape[1]
    h  = 0.5 * np.log((2 * np.pi * np.e)**d * det(np.cov(x.T)))
    return h
def get_mi_mvn(x, y):
    """
    Computes the mutual information I between two multivariate normal random
    variables, X and Y:
    I(X, Y) = H(X) + H(Y) - H(X, Y)
    Arguments:
    ----------
    x, y: (n, d) ndarrays
        n samples from d-dimensional multivariate normal distributions
    Returns:
    --------
    mi: float
        mutual information I(X, Y)
    """

    d = x.shape[1]

    # hx  = 0.5 * log((2 * np.pi * np.e)**d     * det(np.cov(x.T)))
    # hy  = 0.5 * log((2 * np.pi * np.e)**d     * det(np.cov(y.T)))
    # hxy = 0.5 * log((2 * np.pi * np.e)**(2*d) * det(np.cov(x.T, y=y.T)))
    # mi = hx + hy - hxy

    # hx  = 0.5 * log(det(2*np.pi*np.e*np.cov(x.T)))
    # hy  = 0.5 * log(det(2*np.pi*np.e*np.cov(y.T)))
    # hxy = 0.5 * log(det(2*np.pi*np.e*np.cov(np.c_[x,y].T)))
    hx  = get_h_mvn(x)
    hy  = get_h_mvn(y)
    hxy = get_h_mvn(np.c_[x,y])
    mi = hx + hy - hxy

    # mi = 0.5 * (log(det(np.cov(x.T))) + log(det(np.cov(y.T))) - log(det(np.cov(np.c_[x,y].T))))

    return mi

N=900000
dimension = 140
mean  = np.zeros(dimension)
sigma = np.ones((dimension,dimension)) * 0.9
np.fill_diagonal(sigma,1.0)

temp  = np.random.multivariate_normal(mean,sigma,10000)
x_sample = temp[:,:dimension//2]
y_sample = temp[:,dimension//2:]
mi = get_mi_mvn(x_sample,y_sample)
print(temp.shape)
print(mi)

(10000, 140)
3.122075475590634


In [34]:
# layers
def tf_relu(x):   return tf.nn.relu(x)
def d_tf_relu(x): return tf.cast(tf.greater(x,0),tf.float32)

# Func: Fully Connected Layer
class FNN():

    def __init__(self,inc,outc,act=tf_relu,d_act=d_tf_relu,special_init=False,which_reg=0.0):
        if special_init:
            interval = np.sqrt(6.0 / (inc + outc + 1.0))
            self.w = tf.Variable(tf.random_uniform(shape=(inc, outc),minval=-interval,maxval=interval,dtype=tf.float32,seed=2))
            self.b = tf.Variable(tf.random_uniform(shape=(outc),minval=-interval,maxval=interval,dtype=tf.float32,seed=2))
        else:
            self.w = tf.Variable(tf.random_normal([inc,outc], stddev=0.05,seed=2,dtype=tf.float32))
            self.b = tf.Variable(tf.random_normal([outc], stddev=0.05,seed=2,dtype=tf.float32))

        self.m,self.v = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.m_b,self.v_b = tf.Variable(tf.zeros_like(self.b)),tf.Variable(tf.zeros_like(self.b))
        self.act,self.d_act = act,d_act
        self.which_reg = which_reg

    def getw(self): return self.w

    def feedforward(self,input=None):
        self.input = input
        self.layer = tf.matmul(input,self.w) + self.b
        self.layerA = self.act(self.layer)
        return self.layerA

    def backprop(self,gradient=None,which_reg=0):
        grad_part_1 = gradient
        grad_part_2 = self.d_act(self.layer)
        grad_part_3 = self.input

        grad_middle = grad_part_1 * grad_part_2
        grad  = tf.matmul(tf.transpose(grad_part_3),grad_middle)/batch_size
        grad_pass = tf.matmul(grad_middle,tf.transpose(self.w))

        update_w = []

        # Update the Weight First
        update_w.append(tf.assign( self.m,self.m*beta1 + (1-beta1) * (grad)   ))
        update_w.append(tf.assign( self.v,self.v*beta2 + (1-beta2) * (grad ** 2)   ))
        m_hat = self.m / (1-beta1)
        v_hat = self.v / (1-beta2)
        adam_middle = m_hat *  learning_rate/(tf.sqrt(v_hat) + adam_e)
        update_w.append(tf.assign(self.w,tf.subtract(self.w,adam_middle )))

        return grad_pass,update_w

In [20]:
# define layers
# sess = tf.InteractiveSession()
n_hidden = 10 
l1 = FNN(dimension//2,10)
l2 = FNN(10,1)


In [51]:
# create graph
x  = tf.placeholder(tf.float32, [None,dimension//2])
y  = tf.placeholder(tf.float32, [None,dimension//2])
y_ = tf.placeholder(tf.float32, [None,dimension//2])

Wx = tf.Variable(tf.random_normal(stddev=0.1,shape=[dimension//2,n_hidden]))
Wy = tf.Variable(tf.random_normal(stddev=0.1,shape=[dimension//2,n_hidden]))
Wout = tf.Variable(tf.random_normal(stddev=0.1,shape=[n_hidden,1]))

hidden_joint  = tf.matmul(x,Wx)+tf.matmul(y,Wy)
hidden_marg   = tf.matmul(x,Wx)+tf.matmul(y_,Wy)
hidden_jointa = tf_relu(hidden_joint)
hidden_marga  = tf_relu(hidden_marg)
out_joint = tf.matmul(hidden_joint,Wout)
out_marg  = tf.matmul(hidden_marg,Wout)
 
lower_bound=-(tf.reduce_mean(out_joint)-tf.log(tf.reduce_mean(tf.exp(out_marg))))

dwout         = tf.reduce_mean(-((1/N)*tf.transpose(hidden_joint)+(-N)*tf.transpose(hidden_marg)),1,True)
dhidden_joint = -(1/N) * tf.transpose(Wout) * d_tf_relu(hidden_joint)
dhidden_marg  = (-N)*tf.transpose(Wout)     * d_tf_relu(hidden_marg)
dWx = tf.transpose(x) @ dhidden_joint + tf.transpose(x) @ dhidden_marg 
dYx = tf.transpose(y) @ dhidden_joint + tf.transpose(y_) @ dhidden_marg 

updatewX = Wx.assign(Wx-0.000001 * dWx)
updatewY = Wy.assign(Wy-0.000001 * dYx)
updatewO = Wout.assign(Wout-0.000001 * dwout)
update_w = [updatewX,updatewY,updatewO]

In [52]:
values = []
# sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(1000):
    #x_sample  = np.random.normal(0.,sig1,[N,1])
    #y_sample  = np.random.normal(0.,sig2,[N,1])
    y_shuffle = np.random.permutation(y_sample)
    number,_  = sess.run([lower_bound,update_w], feed_dict={x:x_sample,y:y_sample,y_:y_shuffle})
    sys.stdout.write("\n" + str(number))
    sys.stdout.flush()
    values.append(number)
    
plt.plot(values)
plt.plot(np.ones_like(values)*mi)
plt.show()


0.005233477
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan

KeyboardInterrupt: 

In [13]:
! git all-go

[master 6c7fca3] commit
 2 files changed, 10 insertions(+), 22 deletions(-)
Counting objects: 4, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 488 bytes | 488.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/JaeDukSeo/Mututal-Information.git
   0b24b0d..6c7fca3  master -> master
